In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt

In [5]:
pwr_df = pd.read_csv("D:/Nirmal/2022 FALL MIT/RA Documents/GenX/GenX_CO2/GenX/paper_runs/2zone_greenfields/fusion/2001/Results/fusion/fusion_time_10.csv")

In [6]:
net_cap = 12633.69582532336 * 0.7239130435 * 8760
pwr_produced = pwr_df['Net Electric'].sum()

print(pwr_produced/net_cap)


0.7490701764928838
